# LSTM (Long Short Term Memories)
For all pollen types only

## Libraries required

In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import model_from_json

/Users/bruno.wozniak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Do the whole thing

In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

all_pol_wu_df = read_csv('../analyze/pollen_weather.csv')
for pollen in ['Ambrosia', 'Artemisia', 'Asteraceae','Alnus', 'Betula', 'Ericaceae', 'Carpinus', 'Castanea', 'Quercus','Chenopodium', 'Cupressaceae', 'Acer', 'Fraxinus', 'Gramineae', 'Fagus','Juncaceae', 'Aesculus', 'Larix', 'Corylus', 'Juglans', 'Umbellifereae','Ulmus', 'Urtica', 'Rumex', 'Populus', 'Pinaceae', 'Plantago','Platanus', 'Salix', 'Cyperaceae', 'Filipendula', 'Sambucus', 'Tilia']:
    pol_wu_df = all_pol_wu_df[['Date',pollen,'DayOfYear','TempMax','HumidMin','VisibilityAvg']]
    pol_wu_df.set_index('Date',inplace=True)
    values = pol_wu_df.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # How many days to lag + how many features
    n_days = 7
    n_features = 4
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n_days, 1)
    # drop columns we don't want to predict
    reframed.drop(reframed.columns[[6,7,8,9]], axis=1, inplace=True)
    # split into train and test sets
    values = reframed.values
    n_train_days = 18*365
    train = values[:n_train_days, :]
    test = values[n_train_days:, :]
    # split into input and outputs
    n_obs = n_days * n_features
    train_X, train_y = train[:, :n_obs], train[:, -n_features]
    test_X, test_y = test[:, :n_obs], test[:, -n_features]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
    # design network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=100, batch_size=365, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # plot history
#     pyplot.plot(history.history['loss'], label='train')
#     pyplot.plot(history.history['val_loss'], label='test')
#     pyplot.legend()
#     pyplot.show()
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], n_days*n_features))
    # invert scaling for forecast
    inv_yhat = concatenate((yhat, test_X[:, -4:]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, -4:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('[%s] Test RMSE: %.3f' % (pollen, rmse))

    # serialize model to JSON
    model_json = model.to_json()
    with open('model_'+pollen+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('LSTM_model_'+pollen+'.h5')

Train on 6570 samples, validate on 1220 samples
Epoch 1/100
 - 1s - loss: 0.2679 - val_loss: 0.2138
Epoch 2/100
 - 0s - loss: 0.1661 - val_loss: 0.1681
Epoch 3/100
 - 0s - loss: 0.1424 - val_loss: 0.1370
Epoch 4/100
 - 0s - loss: 0.1060 - val_loss: 0.0832
Epoch 5/100
 - 0s - loss: 0.0716 - val_loss: 0.0688
Epoch 6/100
 - 0s - loss: 0.0634 - val_loss: 0.0617
Epoch 7/100
 - 0s - loss: 0.0608 - val_loss: 0.0607
Epoch 8/100
 - 0s - loss: 0.0598 - val_loss: 0.0611
Epoch 9/100
 - 0s - loss: 0.0591 - val_loss: 0.0580
Epoch 10/100
 - 0s - loss: 0.0560 - val_loss: 0.0564
Epoch 11/100
 - 0s - loss: 0.0546 - val_loss: 0.0556
Epoch 12/100
 - 0s - loss: 0.0535 - val_loss: 0.0542
Epoch 13/100
 - 0s - loss: 0.0521 - val_loss: 0.0533
Epoch 14/100
 - 0s - loss: 0.0512 - val_loss: 0.0518
Epoch 15/100
 - 0s - loss: 0.0495 - val_loss: 0.0504
Epoch 16/100
 - 0s - loss: 0.0480 - val_loss: 0.0492
Epoch 17/100
 - 0s - loss: 0.0466 - val_loss: 0.0477
Epoch 18/100
 - 0s - loss: 0.0450 - val_loss: 0.0459
Epoch 1

Epoch 54/100
 - 0s - loss: 0.0094 - val_loss: 0.0097
Epoch 55/100
 - 0s - loss: 0.0076 - val_loss: 0.0085
Epoch 56/100
 - 0s - loss: 0.0082 - val_loss: 0.0104
Epoch 57/100
 - 0s - loss: 0.0091 - val_loss: 0.0104
Epoch 58/100
 - 0s - loss: 0.0091 - val_loss: 0.0098
Epoch 59/100
 - 0s - loss: 0.0086 - val_loss: 0.0091
Epoch 60/100
 - 0s - loss: 0.0076 - val_loss: 0.0100
Epoch 61/100
 - 0s - loss: 0.0075 - val_loss: 0.0090
Epoch 62/100
 - 0s - loss: 0.0075 - val_loss: 0.0076
Epoch 63/100
 - 0s - loss: 0.0077 - val_loss: 0.0077
Epoch 64/100
 - 0s - loss: 0.0066 - val_loss: 0.0096
Epoch 65/100
 - 0s - loss: 0.0084 - val_loss: 0.0087
Epoch 66/100
 - 0s - loss: 0.0079 - val_loss: 0.0085
Epoch 67/100
 - 0s - loss: 0.0078 - val_loss: 0.0076
Epoch 68/100
 - 0s - loss: 0.0065 - val_loss: 0.0084
Epoch 69/100
 - 0s - loss: 0.0075 - val_loss: 0.0093
Epoch 70/100
 - 0s - loss: 0.0085 - val_loss: 0.0103
Epoch 71/100
 - 0s - loss: 0.0069 - val_loss: 0.0074
Epoch 72/100
 - 0s - loss: 0.0062 - val_loss: 

Epoch 6/100
 - 0s - loss: 0.0675 - val_loss: 0.0675
Epoch 7/100
 - 0s - loss: 0.0629 - val_loss: 0.0646
Epoch 8/100
 - 0s - loss: 0.0615 - val_loss: 0.0647
Epoch 9/100
 - 0s - loss: 0.0610 - val_loss: 0.0628
Epoch 10/100
 - 0s - loss: 0.0589 - val_loss: 0.0606
Epoch 11/100
 - 0s - loss: 0.0562 - val_loss: 0.0592
Epoch 12/100
 - 0s - loss: 0.0552 - val_loss: 0.0578
Epoch 13/100
 - 0s - loss: 0.0537 - val_loss: 0.0564
Epoch 14/100
 - 0s - loss: 0.0520 - val_loss: 0.0548
Epoch 15/100
 - 0s - loss: 0.0503 - val_loss: 0.0531
Epoch 16/100
 - 0s - loss: 0.0486 - val_loss: 0.0511
Epoch 17/100
 - 0s - loss: 0.0463 - val_loss: 0.0488
Epoch 18/100
 - 0s - loss: 0.0438 - val_loss: 0.0465
Epoch 19/100
 - 0s - loss: 0.0413 - val_loss: 0.0441
Epoch 20/100
 - 0s - loss: 0.0388 - val_loss: 0.0407
Epoch 21/100
 - 0s - loss: 0.0358 - val_loss: 0.0374
Epoch 22/100
 - 0s - loss: 0.0326 - val_loss: 0.0368
Epoch 23/100
 - 0s - loss: 0.0272 - val_loss: 0.0319
Epoch 24/100
 - 0s - loss: 0.0258 - val_loss: 0.02

Epoch 60/100
 - 0s - loss: 0.0145 - val_loss: 0.0131
Epoch 61/100
 - 0s - loss: 0.0144 - val_loss: 0.0147
Epoch 62/100
 - 0s - loss: 0.0115 - val_loss: 0.0105
Epoch 63/100
 - 0s - loss: 0.0103 - val_loss: 0.0128
Epoch 64/100
 - 0s - loss: 0.0108 - val_loss: 0.0107
Epoch 65/100
 - 0s - loss: 0.0102 - val_loss: 0.0116
Epoch 66/100
 - 0s - loss: 0.0086 - val_loss: 0.0113
Epoch 67/100
 - 0s - loss: 0.0099 - val_loss: 0.0116
Epoch 68/100
 - 0s - loss: 0.0099 - val_loss: 0.0125
Epoch 69/100
 - 0s - loss: 0.0095 - val_loss: 0.0111
Epoch 70/100
 - 0s - loss: 0.0094 - val_loss: 0.0122
Epoch 71/100
 - 0s - loss: 0.0094 - val_loss: 0.0092
Epoch 72/100
 - 0s - loss: 0.0088 - val_loss: 0.0092
Epoch 73/100
 - 0s - loss: 0.0092 - val_loss: 0.0089
Epoch 74/100
 - 0s - loss: 0.0075 - val_loss: 0.0107
Epoch 75/100
 - 0s - loss: 0.0076 - val_loss: 0.0094
Epoch 76/100
 - 0s - loss: 0.0076 - val_loss: 0.0108
Epoch 77/100
 - 0s - loss: 0.0076 - val_loss: 0.0086
Epoch 78/100
 - 0s - loss: 0.0073 - val_loss: 

Epoch 12/100
 - 0s - loss: 0.0531 - val_loss: 0.0548
Epoch 13/100
 - 0s - loss: 0.0521 - val_loss: 0.0531
Epoch 14/100
 - 0s - loss: 0.0502 - val_loss: 0.0520
Epoch 15/100
 - 0s - loss: 0.0492 - val_loss: 0.0500
Epoch 16/100
 - 0s - loss: 0.0470 - val_loss: 0.0484
Epoch 17/100
 - 0s - loss: 0.0455 - val_loss: 0.0466
Epoch 18/100
 - 0s - loss: 0.0434 - val_loss: 0.0449
Epoch 19/100
 - 0s - loss: 0.0415 - val_loss: 0.0436
Epoch 20/100
 - 0s - loss: 0.0395 - val_loss: 0.0416
Epoch 21/100
 - 0s - loss: 0.0374 - val_loss: 0.0386
Epoch 22/100
 - 0s - loss: 0.0349 - val_loss: 0.0355
Epoch 23/100
 - 0s - loss: 0.0320 - val_loss: 0.0346
Epoch 24/100
 - 0s - loss: 0.0292 - val_loss: 0.0327
Epoch 25/100
 - 0s - loss: 0.0272 - val_loss: 0.0306
Epoch 26/100
 - 0s - loss: 0.0260 - val_loss: 0.0292
Epoch 27/100
 - 0s - loss: 0.0248 - val_loss: 0.0279
Epoch 28/100
 - 0s - loss: 0.0237 - val_loss: 0.0268
Epoch 29/100
 - 0s - loss: 0.0229 - val_loss: 0.0258
Epoch 30/100
 - 0s - loss: 0.0216 - val_loss: 

...

[Pinaceae] Test RMSE: 125.294
Train on 6570 samples, validate on 1220 samples
Epoch 1/100
 - 3s - loss: 0.2330 - val_loss: 0.1979
Epoch 2/100
 - 0s - loss: 0.1529 - val_loss: 0.1545
Epoch 3/100
 - 0s - loss: 0.1289 - val_loss: 0.1219
Epoch 4/100
 - 0s - loss: 0.0918 - val_loss: 0.0693
Epoch 5/100
 - 0s - loss: 0.0604 - val_loss: 0.0608
Epoch 6/100
 - 0s - loss: 0.0550 - val_loss: 0.0562
Epoch 7/100
 - 0s - loss: 0.0538 - val_loss: 0.0555
Epoch 8/100
 - 0s - loss: 0.0521 - val_loss: 0.0537
Epoch 9/100
 - 0s - loss: 0.0501 - val_loss: 0.0514
Epoch 10/100
 - 0s - loss: 0.0478 - val_loss: 0.0492
Epoch 11/100
 - 0s - loss: 0.0456 - val_loss: 0.0470
Epoch 12/100
 - 0s - loss: 0.0434 - val_loss: 0.0448
Epoch 13/100
 - 0s - loss: 0.0412 - val_loss: 0.0422
Epoch 14/100
 - 0s - loss: 0.0386 - val_loss: 0.0390
Epoch 15/100
 - 0s - loss: 0.0358 - val_loss: 0.0357
Epoch 16/100
 - 0s - loss: 0.0327 - val_loss: 0.0332
Epoch 17/100
 - 0s - loss: 0.0289 - val_loss: 0.0294
Epoch 18/100
 - 0s - loss: 0.0

Epoch 53/100
 - 0s - loss: 0.0111 - val_loss: 0.0134
Epoch 54/100
 - 0s - loss: 0.0118 - val_loss: 0.0132
Epoch 55/100
 - 0s - loss: 0.0120 - val_loss: 0.0108
Epoch 56/100
 - 0s - loss: 0.0127 - val_loss: 0.0148
Epoch 57/100
 - 0s - loss: 0.0121 - val_loss: 0.0112
Epoch 58/100
 - 0s - loss: 0.0093 - val_loss: 0.0114
Epoch 59/100
 - 0s - loss: 0.0092 - val_loss: 0.0115
Epoch 60/100
 - 0s - loss: 0.0086 - val_loss: 0.0098
Epoch 61/100
 - 0s - loss: 0.0110 - val_loss: 0.0122
Epoch 62/100
 - 0s - loss: 0.0111 - val_loss: 0.0116
Epoch 63/100
 - 0s - loss: 0.0108 - val_loss: 0.0116
Epoch 64/100
 - 0s - loss: 0.0106 - val_loss: 0.0112
Epoch 65/100
 - 0s - loss: 0.0100 - val_loss: 0.0117
Epoch 66/100
 - 0s - loss: 0.0104 - val_loss: 0.0109
Epoch 67/100
 - 0s - loss: 0.0100 - val_loss: 0.0097
Epoch 68/100
 - 0s - loss: 0.0082 - val_loss: 0.0103
Epoch 69/100
 - 0s - loss: 0.0080 - val_loss: 0.0090
Epoch 70/100
 - 0s - loss: 0.0095 - val_loss: 0.0129
Epoch 71/100
 - 0s - loss: 0.0147 - val_loss: 

Epoch 5/100
 - 0s - loss: 0.0870 - val_loss: 0.0800
Epoch 6/100
 - 0s - loss: 0.0693 - val_loss: 0.0667
Epoch 7/100
 - 0s - loss: 0.0637 - val_loss: 0.0637
Epoch 8/100
 - 0s - loss: 0.0616 - val_loss: 0.0620
Epoch 9/100
 - 0s - loss: 0.0598 - val_loss: 0.0602
Epoch 10/100
 - 0s - loss: 0.0586 - val_loss: 0.0583
Epoch 11/100
 - 0s - loss: 0.0570 - val_loss: 0.0567
Epoch 12/100
 - 0s - loss: 0.0554 - val_loss: 0.0553
Epoch 13/100
 - 0s - loss: 0.0537 - val_loss: 0.0537
Epoch 14/100
 - 0s - loss: 0.0518 - val_loss: 0.0516
Epoch 15/100
 - 0s - loss: 0.0495 - val_loss: 0.0493
Epoch 16/100
 - 0s - loss: 0.0469 - val_loss: 0.0469
Epoch 17/100
 - 0s - loss: 0.0444 - val_loss: 0.0444
Epoch 18/100
 - 0s - loss: 0.0417 - val_loss: 0.0420
Epoch 19/100
 - 0s - loss: 0.0384 - val_loss: 0.0388
Epoch 20/100
 - 0s - loss: 0.0345 - val_loss: 0.0349
Epoch 21/100
 - 0s - loss: 0.0325 - val_loss: 0.0317
Epoch 22/100
 - 0s - loss: 0.0292 - val_loss: 0.0301
Epoch 23/100
 - 0s - loss: 0.0279 - val_loss: 0.028

Epoch 59/100
 - 0s - loss: 0.0074 - val_loss: 0.0081
Epoch 60/100
 - 0s - loss: 0.0076 - val_loss: 0.0075
Epoch 61/100
 - 0s - loss: 0.0079 - val_loss: 0.0084
Epoch 62/100
 - 0s - loss: 0.0078 - val_loss: 0.0080
Epoch 63/100
 - 0s - loss: 0.0079 - val_loss: 0.0080
Epoch 64/100
 - 0s - loss: 0.0062 - val_loss: 0.0073
Epoch 65/100
 - 0s - loss: 0.0065 - val_loss: 0.0072
Epoch 66/100
 - 0s - loss: 0.0073 - val_loss: 0.0076
Epoch 67/100
 - 0s - loss: 0.0067 - val_loss: 0.0079
Epoch 68/100
 - 0s - loss: 0.0074 - val_loss: 0.0073
Epoch 69/100
 - 0s - loss: 0.0081 - val_loss: 0.0094
Epoch 70/100
 - 0s - loss: 0.0062 - val_loss: 0.0071
Epoch 71/100
 - 0s - loss: 0.0080 - val_loss: 0.0073
Epoch 72/100
 - 0s - loss: 0.0077 - val_loss: 0.0088
Epoch 73/100
 - 0s - loss: 0.0078 - val_loss: 0.0096
Epoch 74/100
 - 0s - loss: 0.0080 - val_loss: 0.0078
Epoch 75/100
 - 0s - loss: 0.0073 - val_loss: 0.0085
Epoch 76/100
 - 0s - loss: 0.0078 - val_loss: 0.0083
Epoch 77/100
 - 0s - loss: 0.0061 - val_loss: 

Epoch 11/100
 - 0s - loss: 0.0520 - val_loss: 0.0538
Epoch 12/100
 - 0s - loss: 0.0504 - val_loss: 0.0524
Epoch 13/100
 - 0s - loss: 0.0489 - val_loss: 0.0513
Epoch 14/100
 - 0s - loss: 0.0470 - val_loss: 0.0495
Epoch 15/100
 - 0s - loss: 0.0447 - val_loss: 0.0469
Epoch 16/100
 - 0s - loss: 0.0421 - val_loss: 0.0438
Epoch 17/100
 - 0s - loss: 0.0395 - val_loss: 0.0403
Epoch 18/100
 - 0s - loss: 0.0362 - val_loss: 0.0366
Epoch 19/100
 - 0s - loss: 0.0328 - val_loss: 0.0343
Epoch 20/100
 - 0s - loss: 0.0280 - val_loss: 0.0294
Epoch 21/100
 - 0s - loss: 0.0247 - val_loss: 0.0276
Epoch 22/100
 - 0s - loss: 0.0234 - val_loss: 0.0248
Epoch 23/100
 - 0s - loss: 0.0212 - val_loss: 0.0242
Epoch 24/100
 - 0s - loss: 0.0180 - val_loss: 0.0215
Epoch 25/100
 - 0s - loss: 0.0170 - val_loss: 0.0225
Epoch 26/100
 - 0s - loss: 0.0159 - val_loss: 0.0187
Epoch 27/100
 - 0s - loss: 0.0158 - val_loss: 0.0196
Epoch 28/100
 - 0s - loss: 0.0141 - val_loss: 0.0170
Epoch 29/100
 - 0s - loss: 0.0155 - val_loss: 